
# Practicum II - Movie Database Applicaton
### Collaborators: Abasamia Akpan, Rebecca Brent and Alessia Pizzoccheri

This Notebook provides context, documentation and sample code for a Database Application using IMBd datasets; for clarity, this document has been divided into the following sections:

### 1. Introduction
### 2. Assumptions
### 3. Logical Models
### 4. Database and Relations Definition¶
### 1. Introduction
#### 1.1. Looking at the data
In this section, we analyze and deconstruct the various data sets found on the IMBd page; data is organized in seven TSV files, specifically:

Title AKAs
Title Basics
Title Episodes
Title Ratings
Title Crew
Name Basics
Title Principles Our first step is to establish relationships between each data set and identify a primary key (PK) on each table. Since no information was provided regarding the role of columns, we devised a solution to find unique identifiers within each data set; that is, for each table we compare the total number of rows against the total number of unique values for a given column.
1.2 Establishing PKs
Following the system aforementioned, we identified the following primary keys

In [1]:

import pandas as pd
import numpy as np
import os
import sys
import pip
pip.main(['install','prettytable'])
pip.main(['install','mysql-connector-python'])
pip.main(['install','tqdm'])
pip.main(['install','pymysql'])

from tqdm import tqdm



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached prettytable-1.0.1-py2.py3-none-any.whl (22 kB)


ERROR: ipython-sql 0.4.0 has requirement prettytable<1, but you'll have prettytable 1.0.1 which is incompatible.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached PyMySQL-0.10.1-py2.py3-none-any.whl (47 kB)


In [26]:
title_akas = pd.read_csv('title.akas.tsv', delimiter='\t',encoding='utf-8')


/Users/apizzoccheri/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [27]:
# read title basic file
title_basics = pd.read_csv('title.basics.tsv', delimiter='\t',encoding='utf-8')

/Users/apizzoccheri/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
title_episodes = pd.read_csv('title.episode.tsv', delimiter='\t',encoding='utf-8')

In [29]:
title_ratings = pd.read_csv('title.ratings.tsv', delimiter='\t',encoding='utf-8')

In [30]:
name_basics = pd.read_csv('name.basics.tsv', delimiter='\t',encoding='utf-8')

In [31]:
title_principals = pd.read_csv('title.principals.tsv', delimiter='\t',encoding='utf-8')

In [32]:
title_crew = pd.read_csv('title.crew.tsv', delimiter='\t',encoding='utf-8')


In [2]:
import mysql.connector
from prettytable import PrettyTable 
from prettytable import from_csv
from mysql.connector import Error

def create_connection_nodb(host_name, user_name, user_password):
    """ create a connection to mysql host
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_database(connection, db_name):
    """ create a database 
    :param connection: Connection object
    :param db_name: name of database in form of string
    :return:
    """
    cursor = connection.cursor()
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + db_name
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(host_name, user_name, user_password, db_name):
    """ create a connection to the (possibly newly created) database
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :param db_name: string name of database 
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        table_name = create_table_sql.split(" ")[5].split("(")[0]
        print("Successfully created table: " + table_name)
    except Error as e:
        print(e)
        
def main():
    
    database = "imbd_application"
    host_name = "localhost"
    user_name = "root"
    user_password = "alupizzo92"
    
    create_table_titlebasics = """CREATE TABLE IF NOT EXISTS titlebasics(
                                    tconst VARCHAR(1000) PRIMARY KEY,
                                    titleType VARCHAR(1000),
                                    primaryTitle VARCHAR(1000),
                                    originalTitle VARCHAR(1000),
                                    isAdult BOOLEAN,
                                    startYear DATE,
                                    endYear DATE,
                                    runTimeMinutes TIME)"""
    
    create_table_titleratings = """CREATE TABLE IF NOT EXISTS titleratings(
                                    tconst VARCHAR(1000) PRIMARY KEY,
                                    averageRating FLOAT,
                                    numVotes INT,
                                    FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""


    create_table_titleepisodes = """CREATE TABLE IF NOT EXISTS titleepisodes(
                                    tconst VARCHAR(1000) PRIMARY KEY,
                                    seasonNumber INT,
                                    episodeNumber INT,
                                    parentTconst VARCHAR(1000),
                                    FOREIGN KEY (parentTconst) REFERENCES titlebasics(tconst))"""

    create_table_titleAKAs = """CREATE TABLE IF NOT EXISTS titleAKAs(
                                AKAsID INT AUTO_INCREMENT PRIMARY KEY,
                                titleID VARCHAR(1000),
                                ordering INT,
                                title VARCHAR(1000),
                                region VARCHAR(1000),
                                language VARCHAR(1000),
                                isoriginaltitle BOOLEAN,
                                FOREIGN KEY (titleID) REFERENCES titlebasics(tconst))"""

    create_table_namebasics = """CREATE TABLE IF NOT EXISTS namebasics(
                                nconst VARCHAR(1000) PRIMARY KEY,
                                primaryname VARCHAR(1000),
                                birthyear DATE,
                                deathyear DATE,
                                noofmovies INT,
                                age INT,
                                currentdate DATE)"""
    
    create_table_principals = """CREATE TABLE IF NOT EXISTS principals(
                                principalsID INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(1000),
                                ordering INT,
                                category VARCHAR(1000),
                                job VARCHAR(1000),
                                nconst VARCHAR(1000),
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (nconst) REFERENCES namebasics(nconst))"""
    
    create_table_titlecrew = """CREATE TABLE IF NOT EXISTS titlecrew(
                                crewid INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(1000) UNIQUE,
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""
    
    create_table_characters = """CREATE TABLE IF NOT EXISTS characters(
                                characterID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(1000) UNIQUE)"""
    
    create_table_writers = """CREATE TABLE IF NOT EXISTS writers(
                                writerID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(1000) UNIQUE)"""
    
    create_table_directors = """CREATE TABLE IF NOT EXISTS directors(
                                directorID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(1000) UNIQUE)"""
    
    create_table_genres = """CREATE TABLE IF NOT EXISTS genres(
                                genreID INT AUTO_INCREMENT  PRIMARY KEY,
                                genre VARCHAR(1000) UNIQUE)"""
    
    create_table_knownfortitles = """CREATE TABLE IF NOT EXISTS knownfortitles(
                                knownfortitlesID INT AUTO_INCREMENT PRIMARY KEY,
                                titles VARCHAR(1000) UNIQUE)"""
    
    create_table_primaryprofession = """CREATE TABLE IF NOT EXISTS primaryprofession(
                                primaryprofessionID INT AUTO_INCREMENT PRIMARY KEY,
                                profession VARCHAR(1000) UNIQUE)"""
    
    create_table_types = """CREATE TABLE IF NOT EXISTS types(
                                typeID INT AUTO_INCREMENT  PRIMARY KEY,
                                type VARCHAR(1000) UNIQUE)"""
    
    create_table_attributes = """CREATE TABLE IF NOT EXISTS attributes(
                                attributeID INT AUTO_INCREMENT PRIMARY KEY,
                                attribute VARCHAR(1000) UNIQUE)"""
    
    create_table_genretitlebasics = """CREATE TABLE IF NOT EXISTS genretitlebasics(
                                titlebasicsID VARCHAR(1000),
                                genreID INT,
                                PRIMARY KEY (titlebasicsID, genreID),
                                FOREIGN KEY (titlebasicsID) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (genreID) REFERENCES genres(genreID))"""
    
    create_table_crewdirectors = """CREATE TABLE IF NOT EXISTS crewdirectors(
                                crewid INT,
                                directorid INT,
                                PRIMARY KEY (crewid ,directorid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (directorid) REFERENCES directors(directorID))"""
    
    create_table_crewwriters = """CREATE TABLE IF NOT EXISTS crewwriters(
                                crewid INT,
                                writerid INT,
                                PRIMARY KEY (crewid, writerid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (writerid) REFERENCES writers(writerid))"""
    
    create_table_principalscharacters = """CREATE TABLE IF NOT EXISTS principalscharacters(
                                principalsid INT,
                                characterid INT,
                                PRIMARY KEY (principalsid, characterid),
                                FOREIGN KEY (principalsid) REFERENCES principals(principalsID),
                                FOREIGN KEY (characterid) REFERENCES characters(characterid))"""
    
    create_table_AKAattributes = """CREATE TABLE IF NOT EXISTS AKAattributes(
                                akasid INT,
                                attributeid INT,
                                PRIMARY KEY (akasid, attributeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (attributeid) REFERENCES attributes(attributeid))"""
    
    create_table_AKAtype = """CREATE TABLE IF NOT EXISTS AKAtype(
                                akasid INT,
                                typeid INT,
                                PRIMARY KEY (akasid, typeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (typeid) REFERENCES type(typeid))"""
    
    create_table_primaryprofessiontitlenames = """CREATE TABLE IF NOT EXISTS primaryprofessiontitlenames(
                                primaryprofessionid INT,
                                namebasicsid VARCHAR(255),
                                PRIMARY KEY (primaryprofessionid, namebasicsid),
                                FOREIGN KEY (primaryprofessionid) REFERENCES primaryprofession(primaryprofessionid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst))"""
    
    create_table_knownfortitlenames = """CREATE TABLE IF NOT EXISTS knownfortitlenames(
                                namebasicsid VARCHAR(255),
                                knownfortitlesid INT,
                                PRIMARY KEY (namebasicsid, knownfortitlesid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst),
                                FOREIGN KEY (knownfortitlesid) REFERENCES knownfortitles(knownfortitlesid))"""
    
    conn = create_connection_nodb(host_name, user_name, user_password)
    
    # create database object if not already created and then connect the cursor to it
    if conn is not None:
        create_database(conn, database)
        conn = create_connection(host_name, user_name, user_password, database)
        
        # create tables
        if conn is not None:
            # create Title Basics table
            create_table(conn, create_table_titlebasics)

            # create Title Ratings table
            create_table(conn, create_table_titleratings)
            
            # create title episodes table
            create_table(conn, create_table_titleepisodes)
            
            # create title AKAs table
            create_table(conn, create_table_titleAKAs)
            
            # create name basics table
            create_table(conn, create_table_namebasics)
            
            # create principals table
            create_table(conn, create_table_principals)
            
            # create title crew table
            create_table(conn, create_table_titlecrew)
            
            # create characters table
            create_table(conn,create_table_characters)
            
            # create writers table
            create_table(conn,create_table_writers)
            
            # create directors table
            create_table(conn,create_table_directors)
            
            # create genres table
            create_table(conn,create_table_genres)
            
            # create known for titles table
            create_table(conn,create_table_knownfortitles)
            
            # create primary profession table
            create_table(conn,create_table_primaryprofession)
            
            # create type table
            create_table(conn,create_table_types)
            
            # create attributes table
            create_table(conn,create_table_attributes)
            
            # create genre title basics table
            create_table(conn, create_table_genretitlebasics)
            
            # create crew directors table
            create_table(conn,create_table_crewdirectors)
            
            # create crew writers table
            create_table(conn,create_table_crewwriters)
            
            # create principals characters table
            create_table(conn, create_table_principalscharacters)
            
            # create AKA Attributes table
            create_table(conn, create_table_AKAattributes)
            
            # create AKA Type table
            create_table(conn, create_table_AKAtype)
            
            # create Primary Profession Title Names table
            create_table(conn, create_table_primaryprofessiontitlenames)
            
            # create known for title names table
            create_table(conn, create_table_knownfortitlenames)
                
        else:
            print("Error! cannot create the database connection.")
    
    else:
        print("Error! Cannot create the MySQL connection.")
    
    


if __name__ == '__main__':
    main()


Connection to MySQL DB successful
Database created successfully
Connection to MySQL DB successful
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key 

## SQL communication helper ##

In [55]:
import sqlalchemy
database_username = 'root'
database_password = 'alupizzo92'
database_ip       = 'localhost'
database_name     = 'imbd_application'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

## LOAD DATA IN DATABASE ##


In [57]:
def preprocess_dataframe_title_basics(data_frame):
    title_basics_new_series=data_frame

    b = pd.to_datetime(title_basics_new_series['runtimeMinutes'], errors='coerce', format='%M')


    b.to_frame().runtimeMinutes
    df_1 = pd.DataFrame(dict(time_stamps = b)) 

    df_1['runtimeMinutes'] = df_1['time_stamps'].dt.minute
    title_basics_new_series=title_basics_new_series.drop(['runtimeMinutes'],axis=1)
    title_basics_new_series['runtimeMinutes']=df_1['runtimeMinutes']

    #print(title_basics_new_series)
    
    title_basics_new_frame_transpose=title_basics_new_series.astype({'isAdult': 'int32'})
    

    title_basics_new_frame_transpose["isAdult"]=title_basics_new_frame_transpose.isAdult.mask(title_basics_new_frame_transpose.isAdult > 1,1)
    #g.isAdult=g.isAdult.mask(g.isAdult > 1,1)


    #needs to be fixed
    title_basics_new_frame_transpose['startYear']= pd.to_datetime(title_basics.startYear, errors='coerce', format='%Y')


    
    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]

    
    title_basics_new_frame_transpose['endYear']= pd.to_datetime(title_basics.endYear, errors='coerce', format='%Y')
   # title_basics_new_frame_transpose['endYear']=[time.date() for time in title_basics_new_frame_transpose['endYear']]

    title_basics_new_frame_transpose=title_basics_new_frame_transpose.drop(['genres'],axis=1)
    #print(title_basics_new_frame_transpose)
    
    #title_basics_new_frame_transpose.to_sql('titlebasics',con=database_connection, if_exists='append',index=False)
    new_row = {'tconst':None, 
        'titleType':None, 
        'primaryTitle':None,
        'originalTitle':None,
        'isAdult':None, 
        'startYear':None,
        'endYear':None,
        'runtimeMinutes':None         
       } 
  
    title_basics_new_frame_transpose = title_basics_new_frame_transpose.append(new_row, ignore_index=True)
    return title_basics_new_frame_transpose

title_basics_preprocessed=preprocess_dataframe_title_basics(title_basics)



In [58]:
len(title_basics_preprocessed)
title_basics_preprocessed

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
0,tt0000001,short,Carmencita,Carmencita,0,1894-01-01,NaT,1.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892-01-01,NaT,5.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892-01-01,NaT,4.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892-01-01,NaT,12.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893-01-01,NaT,1.0
...,...,...,...,...,...,...,...,...
7300497,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010-01-01,NaT,NaN
7300498,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010-01-01,NaT,NaN
7300499,tt9916856,short,The Wind,The Wind,0,2015-01-01,NaT,27.0
7300500,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014-01-01,NaT,10.0


In [59]:
title_basics_preprocessed_new=title_basics_preprocessed.rename(columns={"runtimeMinutes": "runTimeMinutes"})

title_basics_preprocessed_new.runTimeMinutes

0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
7300497     NaN
7300498     NaN
7300499    27.0
7300500    10.0
7300501     NaN
Name: runTimeMinutes, Length: 7300502, dtype: float64

In [60]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
    return 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [62]:
#def insert_into_db(data_frame):
batch_size=100000
length=len(df2)//batch_size+1
for chunk in tqdm(batch(df2,batch_size),total=length):
    load_dataframe(chunk,'titlebasics')
     #return

NameError: name 'df2' is not defined

## Title ratings ##

In [63]:
def preprocess_dataframe_title_ratings(data_frame):
    title_ratings_new_series=data_frame
    
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'averageRating': 'float'})
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'numVotes': 'int32'})
  
    return title_ratings_new_frame_avg_rating
title_ratings_preprocessed=preprocess_dataframe_title_ratings(title_ratings)

In [64]:
len(title_ratings_preprocessed)


1086514

In [65]:
batch_size=100000
length=len(title_ratings_preprocessed)//batch_size+1
for chunk in tqdm(batch(title_ratings_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleratings')

100%|██████████| 11/11 [00:15<00:00,  1.39s/it]


## Title Episodes ##

In [84]:
def preprocess_dataframe_title_episodes(data_frame):
    title_episodes_new_series=data_frame
    title_episodes_coerce_seasonNum = pd.to_numeric(title_episodes_new_series['seasonNumber'], errors='coerce')
    title_episodes_coerce_episodeNum = pd.to_numeric(title_episodes_new_series['episodeNumber'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_episodes_coerce_seasonNum)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_episodes_coerce_episodeNum)) 

    df_1['seasonNumber'] = df_1['test_column_1']
    df_2['episodeNumber'] = df_2['test_column_2']
    
    title_episodes_new_frame=title_episodes_new_series.drop(['seasonNumber'],axis=1)
    title_episodes_new_frame=title_episodes_new_series.drop(['episodeNumber'],axis=1)
    
    title_episodes_new_frame['seasonNumber']=df_1['seasonNumber']
    title_episodes_new_frame['episodeNumber']=df_2['episodeNumber']
       
    return title_episodes_new_frame
title_episodes_preprocessed=preprocess_dataframe_title_episodes(title_episodes)

In [85]:
title_episodes_preprocessed

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0
...,...,...,...,...
5252722,tt9916846,tt1289683,3.0,18.0
5252723,tt9916848,tt1289683,3.0,17.0
5252724,tt9916850,tt1289683,3.0,19.0
5252725,tt9916852,tt1289683,3.0,20.0


In [86]:
title_episodes_preprocessed.tconst=title_episodes_preprocessed.tconst.mask(~title_episodes_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)


In [87]:
batch_size=100000
length=len(title_episodes_preprocessed)//batch_size+1
for chunk in tqdm(batch(title_episodes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleepisodes')

100%|██████████| 53/53 [01:18<00:00,  1.49s/it]


## Title Akas ##

In [88]:
def preprocess_dataframe_title_akas(data_frame):
    title_akas_new_series=data_frame
    title_akas_new_series=title_akas_new_series.drop(['types', 'attributes'],axis=1)
    
    title_akas_coerce_ordering = pd.to_numeric(title_akas_new_series['ordering'], errors='coerce')
    title_akas_coerce_isOriginalTitle = pd.to_numeric(title_akas_new_series['isOriginalTitle'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_akas_coerce_ordering)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_akas_coerce_isOriginalTitle)) 

    df_1['ordering'] = df_1['test_column_1']
    df_2['isOriginalTitle'] = df_2['test_column_2']
    
    title_akas_new_frame=title_akas_new_series.drop(['ordering'],axis=1)
    title_akas_new_frame=title_akas_new_series.drop(['isOriginalTitle'],axis=1)
    
    title_akas_new_frame['ordering']=df_1['ordering']
    title_akas_new_frame['isOriginalTitle']=df_2['isOriginalTitle']
    
    
    
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'title': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'language': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'region': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'ordering': 'int32'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'isOriginalTitle': 'int32'})
    


    #print(title_ratings_new_frame_avg_rating)
    #title_ratings_new_frame_avg_rating.to_sql('titleratings',con=database_connection, if_exists='append',index=False)   
    return title_akas_new_frame
title_akas_preprocessed=preprocess_dataframe_title_akas(title_akas)

In [197]:
title_akas_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23944188 entries, 0 to 23944187
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   titleId          object 
 1   ordering         int64  
 2   title            object 
 3   region           object 
 4   language         object 
 5   isOriginalTitle  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ GB


In [89]:
title_akas_preprocessed[~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst)]

,titleId,ordering,title,region,language,isOriginalTitle
75529,tt0021006,1,"Ja, der Himmel über Wien",AT,\N,NaN
78082,tt0021453,1,Tapping Toes,US,\N,NaN
87197,tt0023019,1,Hollywood on Parade,US,\N,NaN
97581,tt0024677,1,Tom's in Town,US,\N,NaN
174103,tt0036165,1,Missing Men,US,\N,NaN
...,...,...,...,...,...,...
20551354,tt7978886,1,State of Defacto,AM,\N,0.0
20936004,tt8206494,1,Blood for Dust,US,\N,0.0
21374750,tt8466868,1,Butaca,ES,\N,0.0
22241328,tt8982514,1,To The Bitter End,US,\N,0.0


In [90]:
title_akas_preprocessed.titleId=title_akas_preprocessed.titleId.mask(~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst),None)



In [91]:
batch_size=100000
length=len(title_akas_preprocessed)//batch_size+1
for chunk in tqdm(batch(title_akas_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleAKAs')

100%|██████████| 240/240 [08:08<00:00,  2.04s/it]


## Name Basics-ncost column name is not in right format-see error below. Tried to convert it into sql syntax no luck ##

In [5]:
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0059956,tt0057345,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0078723,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0083922,tt0050976"
...,...,...,...,...,...,...
10473337,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10473338,nm9993716,Essias Loberg,\N,\N,NaN,\N
10473339,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10473340,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [154]:
import pymysql
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='new_password',
                         db='imdb_new_final')


# create cursor
cursor=connection.cursor()

In [66]:
def preprocess_dataframe_name_basics(data_frame):
    name_basics_new_series=data_frame
    name_basics_new_series=name_basics_new_series.drop(['primaryProfession', 'knownForTitles'],axis=1)
    
    name_basics_new_series['birthYear']= pd.to_datetime(name_basics_new_series.birthYear, errors='coerce', format='%Y')


    
    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]

    
    name_basics_new_series['deathYear']= pd.to_datetime(name_basics_new_series.deathYear, errors='coerce', format='%Y')
    
    
      
    return name_basics_new_series
name_basics_preprocessed=preprocess_dataframe_name_basics(name_basics)

In [67]:
name_basics_preprocessed["noofmovies"] = ""
name_basics_preprocessed["age"] = ""
name_basics_preprocessed["currentdate"] = ""








In [68]:
name_basics_preprocessed_new= pd.to_numeric(name_basics_preprocessed['noofmovies'], errors='coerce')

name_basics_preprocessed_new = pd.to_numeric(name_basics_preprocessed['age'], errors='coerce')
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed.currentdate, errors='coerce', format='%Y')


In [69]:
df_1 = pd.DataFrame(dict(test_column_1 = name_basics_preprocessed_new)) 
df_2 = pd.DataFrame(dict(test_column_2 = name_basics_preprocessed_new)) 

df_1['noofmovies'] = df_1['test_column_1']
df_2['age'] = df_2['test_column_2']

name_basics_preprocessed['noofmovies']=df_1['noofmovies']
name_basics_preprocessed['age']=df_2['age']

In [70]:
name_basics_preprocessed

name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed['currentdate'])


In [71]:
name_basics_preprocessed

,nconst,primaryName,birthYear,deathYear,noofmovies,age,currentdate
0,nm0000001,Fred Astaire,1899-01-01,1987-01-01,NaN,NaN,NaT
1,nm0000002,Lauren Bacall,1924-01-01,2014-01-01,NaN,NaN,NaT
2,nm0000003,Brigitte Bardot,1934-01-01,NaT,NaN,NaN,NaT
3,nm0000004,John Belushi,1949-01-01,1982-01-01,NaN,NaN,NaT
4,nm0000005,Ingmar Bergman,1918-01-01,2007-01-01,NaN,NaN,NaT
...,...,...,...,...,...,...,...
10473337,nm9993714,Romeo del Rosario,NaT,NaT,NaN,NaN,NaT
10473338,nm9993716,Essias Loberg,NaT,NaT,NaN,NaN,NaT
10473339,nm9993717,Harikrishnan Rajan,NaT,NaT,NaN,NaN,NaT
10473340,nm9993718,Aayush Nair,NaT,NaT,NaN,NaN,NaT


In [44]:
type(list(name_basics_preprocessed.columns[0]))

list

In [72]:
#for col in name_basics_preprocessed.columns: 
    #print(str(col).iloc[0])

name_basics_preprocessed_new=name_basics_preprocessed.rename(columns={"nconst": "%snconst"})


In [46]:
name_basics_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10473342 entries, 0 to 10473341
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   nconst       object        
 1   primaryName  object        
 2   birthYear    datetime64[ns]
 3   deathYear    datetime64[ns]
 4   noofmovies   float64       
 5   age          float64       
 6   currentdate  datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(2)
memory usage: 559.3+ MB


## Approach 1- With sql alchemy ##

In [73]:
batch_size=100000
length=len(name_basics_preprocessed)//batch_size+1
for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

100%|██████████| 105/105 [03:41<00:00,  2.11s/it]


In [80]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="alupizzo92",
    database="imbd_application"
)
mycursor = mydb.cursor()

query = "DESCRIBE namebasics; SELECT * FROM namebasics LIMIT 1;"
mycursor.execute(query)

## Approach 2- Without sql alchemy ##

In [206]:
df = df.apply(lambda col: col.astype(str) if col.dtype == 'datetime64[ns]' else col)

batch_size=100000
length=len(name_basics_preprocessed)//batch_size+1
for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    #load_dataframe(chunk,'namebasics')
    cols = "`,`".join([str(i) for i in name_basics_preprocessed.columns.tolist()])

    # Insert DataFrame recrds one by one.
    for i,row in name_basics_preprocessed.iterrows():
        sql = "INSERT INTO `namebasics` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

  0%|          | 0/105 [00:40<?, ?it/s]


AttributeError: 'Timestamp' object has no attribute 'translate'

## Approach 3-with sql alchemy- still failing 

In [227]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
   # engine.execute('INSERT INTO namebasics VALUES (:data_frame_new.nconst, :data_frame_new.primaryName, :data_frame_new.birthYear, :data_frame_new.deathYear, :data_frame_new.noofmovies, :data_frame_new.age, :data_frame_new.currentdate' )
    return 

In [213]:
batch_size=100000
length=len(title_akas_preprocessed)//batch_size+1
for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

  0%|          | 0/240 [00:01<?, ?it/s]


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'nconst' in 'field list'
[SQL: INSERT INTO namebasics (nconst, `primaryName`, `birthYear`, `deathYear`, noofmovies, age, currentdate) VALUES (%(nconst)s, %(primaryName)s, %(birthYear)s, %(deathYear)s, %(noofmovies)s, %(age)s, %(currentdate)s)]
[parameters: ({'nconst': 'nm0000001', 'primaryName': 'Fred Astaire', 'birthYear': datetime.datetime(1899, 1, 1, 0, 0), 'deathYear': datetime.datetime(1987, 1, 1, 0, 0), 'noofmovies': None, 'age': None, 'currentdate': None}, {'nconst': 'nm0000002', 'primaryName': 'Lauren Bacall', 'birthYear': datetime.datetime(1924, 1, 1, 0, 0), 'deathYear': datetime.datetime(2014, 1, 1, 0, 0), 'noofmovies': None, 'age': None, 'currentdate': None}, {'nconst': 'nm0000003', 'primaryName': 'Brigitte Bardot', 'birthYear': datetime.datetime(1934, 1, 1, 0, 0), 'deathYear': None, 'noofmovies': None, 'age': None, 'currentdate': None}, {'nconst': 'nm0000004', 'primaryName': 'John Belus

## Principals-will insert when nconst is in the right format  ##

## Title crew ##

In [33]:
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7303225,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7303226,tt9916850,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
7303227,tt9916852,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
7303228,tt9916856,nm10538645,nm6951431


In [49]:
title_crew_processed=title_crew.drop(['directors'],axis=1)
title_crew_processed=title_crew.drop(['writers'],axis=1)


In [50]:
batch_size=100000
length=len(title_crew_processed)//batch_size+1
for chunk in tqdm(batch(title_crew_processed,batch_size),total=length):
    load_dataframe(chunk,'titlecrew')

NameError: name 'batch' is not defined

## Writers ##

In [239]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
writers_preprocessed=preprocess_dataframe_parsing(title_crew.writers, "writers")

In [228]:
batch_size=100000
length=len(writers_preprocessed)//batch_size+1
for chunk in tqdm(batch(writers_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'writers')

100%|██████████| 121/121 [02:24<00:00,  1.19s/it]


## Directors ##

In [222]:
directors_preprocessed=preprocess_dataframe_parsing(title_crew.directors, "directors")

In [229]:
batch_size=100000
length=len(directors_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'directors')

100%|██████████| 87/87 [01:37<00:00,  1.12s/it]


## Genres ##

In [232]:
genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres")

In [233]:
batch_size=100000
length=len(genres_preprocessed)//batch_size+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

100%|██████████| 116/116 [02:06<00:00,  1.09s/it]


## Known for titles ##

In [235]:
knownForTitles_preprocessed=preprocess_dataframe_parsing(name_basics.knownForTitles, "knownForTitles")

In [236]:
batch_size=100000
length=len(knownForTitles_preprocessed)//batch_size+1
for chunk in tqdm(batch(knownForTitles_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'knownForTitles')

100%|██████████| 184/184 [03:27<00:00,  1.13s/it]


## primary profession ##

In [237]:
pprofession_preprocessed=preprocess_dataframe_parsing(name_basics.primaryProfession, "primaryProfession")

In [238]:
batch_size=100000
length=len(pprofession_preprocessed)//batch_size+1
for chunk in tqdm(batch(pprofession_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'primaryProfession')

100%|██████████| 136/136 [02:23<00:00,  1.05s/it]


## types ## 

In [240]:
types_preprocessed=preprocess_dataframe_parsing(title_akas.types, "types")

In [241]:
batch_size=100000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'types')

100%|██████████| 240/240 [04:11<00:00,  1.05s/it]


## attributes ##

In [242]:
attributes_preprocessed=preprocess_dataframe_parsing(title_akas.attributes, "attributes")

batch_size=100000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'attributes')

## Characters ##

In [247]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(=data_frame[column_name].str.replace('[\[\]"]',"").str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
characters_preprocessed=preprocess_dataframe_parsing(title_principals.characters, "characters")

In [248]:
batch_size=100000
length=len(characters_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'characters')

 27%|██▋       | 116/427 [02:15<06:02,  1.17s/it]
